Code to calculate Life Cycle Impacts with brightway ;)

In [1]:
from brightway2 import *
import pandas as pd
import numpy as np

In [2]:
#Set Project
projects.set_current('metals_v1')

In [17]:
#Import data from excel file
excel_file = "/Users/margueritefauroux/Desktop/data_metals.xlsx"
sheet_name = "metals_ei"
df_metals_ei = pd.read_excel(excel_file, sheet_name=sheet_name)

In [28]:
years = ["2022","2030","2040","2050"]

In [15]:
list(databases)

['biosphere3',
 'cutoff38',
 'biosphere_regionalized_water_flows',
 'cutoff38_regionalized',
 'cutoff38_regio_copy',
 'NZE_2022',
 'NZE_2025',
 'NZE_2030',
 'NZE_2035',
 'NZE_2040',
 'NZE_2045',
 'NZE_2050',
 'SPS_2022',
 'SPS_2025',
 'SPS_2030',
 'SPS_2035',
 'SPS_2040',
 'SPS_2045',
 'SPS_2050']

In [14]:
del databases['metals9']
del databases['metals10']
del databases['metals11']
del databases['SSP1_2050_elec']


Vacuuming database 
Vacuuming database 
Vacuuming database 
Vacuuming database 


In [19]:
#Multiply the quantities by 1e6 to have kilograms instead of kilotons
df_metals_ei[years] *= 1000000

In [20]:
#Demand by metal and by technology for the scenario NZE
df_NZE = df_metals_ei.loc[df_metals_ei['Scenario'] == 'NZE'].copy()

In [44]:
#Demand by metal and by technology for the scenario SPS
df_SPS = df_metals_ei.loc[df_metals_ei['Scenario'] == 'SPS'].copy()

In [22]:
#Total demand by metal
df_NZE_all = df_NZE.groupby('Metal')[years].sum()

In [45]:
#Total demand by metal
df_SPS_all = df_SPS.groupby('Metal')[years].sum()

In [24]:
impact_world_regio = [i for i in methods if 'IMPACT World+ Damage 2.0_regionalized' == i[0]]

In [25]:
impact_mid_regio = [i for i in methods if 'IMPACT World+ Midpoint 2.0_regionalized' == i[0]]

In [26]:
climate = [('IMPACT World+ Midpoint 2.0_regionalized', 'Midpoint', 'Climate change, long term'),
           ('IMPACT World+ Midpoint 2.0_regionalized', 'Midpoint', 'Climate change, short term')]

In [38]:
impact_world_eq_hh = [('IMPACT World+ Damage 2.0_regionalized', 'Human health', 'Total human health'),
                        ('IMPACT World+ Damage 2.0_regionalized', 'Ecosystem quality', 'Total ecosystem quality')]

In [37]:
list(methods)

[('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP infinite'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 500a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 20a'),
 ('CML 2001 (superseded)', 'terrestrial ecotoxicity', 'TAETP 100a'),
 ('CML 2001 (superseded)', 'malodours air', 'malodours air'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 20a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP infinite'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 500a'),
 ('CML 2001 (superseded)', 'human toxicity', 'HTP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 500a'),
 ('CML 2001 (superseded)', 'climate change', 'lower limit of net GWP'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 20a'),
 ('CML 2001 (superseded)', 'climate change', 'GWP 100a'),
 ('CML 2001 (superseded)', 'climate change', 'upper limit of net GWP'),
 ('CML 2001 (superseded)', 'land use', 'competition'),
 ('CML 2001 (superseded)', 'acidification potential'

In [ ]:
#Create a dictionary with all the activities and their keys
activities_dict_NZE = {}

for index, row in df_NZE.iterrows():
    metal = row['Metal']
    market = row['Market']
    product = row['Reference Product']
    location = row['Location']
    
    for year in years:
        activity_name = f"act_{metal.lower()}_{year}"
        search_criteria = f"{market}"
        
        activities = Database('NZE_' + year).search(search_criteria, limit=1000)
        
        filtered_activities = [i for i in activities if i['name'] == search_criteria and i['location'] == location 
                                and i['reference product'] == product]
        
        if filtered_activities:
            activity = filtered_activities[0]
            activities_dict_NZE[activity_name] = activity
            print(f"{activity_name}")
        else:
            print(f"No activity found for {activity_name}")

In [ ]:
#Create a dictionary with all the activities and their keys
activities_dict_SPS = {}

for index, row in df_SPS.iterrows():
    metal = row['Metal']
    market = row['Market']
    product = row['Reference Product']
    location = row['Location']
    
    for year in years:
        activity_name = f"act_{metal.lower()}_{year}"
        search_criteria = f"{market}"
        
        activities = Database('SPS_' + year).search(search_criteria, limit=1000)
        
        filtered_activities = [i for i in activities if i['name'] == search_criteria and i['location'] == location 
                                and i['reference product'] == product]
        
        if filtered_activities:
            activity = filtered_activities[0]
            activities_dict_SPS[activity_name] = activity
            print(f"{activity_name}")
        else:
            print(f"No activity found for {activity_name}")

In [ ]:
#Create the calculation setups by technology and by year
for technology in df_NZE['Technology'].unique():
    df_NZE_by_techno = df_NZE[df_NZE['Technology'] == technology]
    
    for year in years:
        inv = {}

        for metal in df_NZE_by_techno['Metal'].unique():
            quantity = df_NZE_by_techno.loc[df_NZE_by_techno['Metal'] == metal, year].sum()
            activity_name = f"act_{metal.lower()}_{year}"
            if activity_name in activities_dict_NZE:
                inv[activities_dict_NZE[activity_name].key] = quantity
        
        inv_list = []
        for key, value in inv.items():
            inv_list.append({key: value})
    
        calculation_setups[f'NZE_{year}_{technology.lower()}'] = {'inv': inv_list, 'ia': impact_world_regio}
        

In [ ]:
calculation_setups['SPS_2022_ev']['inv']

In [ ]:
#Create a dataframe to store the results
df_results = pd.DataFrame(columns=["Year", "Technology", "Activity", "Scenario"] + [i[2] for i in impact_world_regio])

In [ ]:
#Calculate the impacts by year and by technology
def get_lca(year):
    multilca = MultiLCA("SPS_" + str(year) + "_electricity networks")
    return multilca

row = 0 

for year in years:
    lca = get_lca(year)
    for i, result in enumerate(lca.results):
        activity_name = Database('SPS_' + str(year)).get(list(lca.all.keys())[i][1]).as_dict()['name']
        year_techno_metal = np.array([year, "Electricity Networks", activity_name, "SPS"])
        results = lca.results[i]
        df_results.loc[row] = np.concatenate((year_techno_metal, results))
        row+=1

In [ ]:
#Export the results in an excel file
df_results.to_excel('LCA_results_SPS_networks.xlsx', index=False) 

In [46]:
#Create the calculation setups by year (total demand and not demand by technology)
for year in years:
    inv = {}

    for index,row in df_SPS_all.iterrows():
        quantity = df_SPS_all.loc[index, year].sum()
        activity_name = f"act_{index.lower()}_{year}"
        if activity_name in activities_dict_SPS:
            inv[activities_dict_SPS[activity_name].key] = quantity
        
    inv_list = []
    for key, value in inv.items():
        inv_list.append({key: value})
    
    calculation_setups[f'SPS_{year}_all'] = {'inv': inv_list, 'ia': impact_world_eq_hh}

In [50]:
#Create a dataframe to store the results
df_results_all_SPS = pd.DataFrame(columns=["Year", "Metal"] + [i[2] for i in impact_world_eq_hh])

In [34]:
#Calculate the impacts by year (total demand)
row = 0 

def get_lca(year):
    multilca = MultiLCA("SPS_" + str(year) + "_all")
    return multilca

for year in years:
    lca = get_lca(year)
    for i, result in enumerate(lca.results):
        activity_name = Database('SPS_' + str(year)).get(list(lca.all.keys())[i][1]).as_dict()['name']
        year_metal = np.array([year,activity_name])
        results = lca.results[i]
        df_results_all_SPS.loc[row] = np.concatenate((year_metal, results))
        row+=1
 

In [35]:
#Export the results in an excel file
df_results_all_SPS.to_excel('LCA_results_all_SPS.xlsx', index=False) 